<a href="https://colab.research.google.com/github/andrewpkitchin/parsing-equations/blob/main/introduction_to_postfix_to_binary_expression_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: http://www.cs.man.ac.uk/~pjj/cs212/fix.html

**Postfix notation** (also known as **"Reverse Polish notation"**): X Y +

Operators are written after their operands. The infix expression A * ( B + C ) / D is equivalent to A B C + * D /

The order of evaluation of operators is always left-to-right, and brackets cannot be used to change this order. Because the "+" is to the left of the "*" in the example above, the addition must be performed before the multiplication.
Operators act on values immediately to the left of them. For example, the "+" above uses the "B" and "C". We can add (totally unnecessary) brackets to make this explicit:
( (A (B C +) *) D /)

Thus, the "*" uses the two values immediately preceding: "A", and the result of the addition. Similarly, the "/" uses the result of the multiplication and the "D".

In [7]:
# Stack
class Stack():
    def __init__(self):
        self.items = []

    def push(self, item):
        self.items.append(item)				

    def pop(self):
        return self.items.pop()
    
    def is_empty(self):
        return self.items == []
    
    def peek(self):
        if not self.is_empty():
            return self.items[-1]
        
    def get_stack(self):
        return self.items

    def height(self):
        return len(self.items)

In [8]:
class Node(object):
  def __init__(self, value):
    self.value = value
    self.left = None
    self.right = None
    self.depth = None
    self.pointer = None 

In [47]:
class BinaryTree():
  def __init__(self, root):
    self.root = Node(root)

# Insertion methods

  def insert_right(self, new_val, node):
    if type(new_val) == Node:
      node.right = new_val
    else:
      node.right = Node(new_val)
  
  def insert_left(self, new_val, node):
    if type(new_val) == Node:
      node.left = new_val
    else:
      node.left = Node(new_val)

  def insert_tree_helper(self, start, node):
    if start.left:
      self.insert_left(start.left.value, node)
      self.insert_tree_helper(start.left, node.left)

    if start.right:
      self.insert_right(start.right.value, node)
      self.insert_tree_helper(start.right, node.right)
  
  def tree_insert_left(self, tree, node):
    self.insert_left(tree.root.value, node)
    self.insert_tree_helper(tree.root, node.left)
  
  def tree_insert_right(self, tree, node):
    self.insert_right(tree.root.value, node)
    self.insert_tree_helper(tree.root, node.right)    

# Print methods

  def print_tree(self, traversal_type):
    if traversal_type == "preorder":
        return self.preorder_print(self.root, "")
    elif traversal_type == "inorder":
        return self.inorder_print(self.root, "")
    elif traversal_type == "postorder":
        return self.postorder_print(self.root, "")

    else:
        print("Traversal type " + str(traversal_type) + " is not supported.")
        return False

  def inorder_print(self, start, traversal):
        """Left->Root->Right"""
        if start:
            traversal = self.inorder_print(start.left, traversal)
            traversal += str(start.value)
            traversal = self.inorder_print(start.right, traversal)
        return traversal

  def preorder_print(self, start, traversal):
        """Root->Left->Right"""
        if start:
            traversal += (str(start.value))
            print("1", traversal)
            traversal = self.preorder_print(start.left, traversal)
            print("2", traversal)
            traversal = self.preorder_print(start.right, traversal)
            print("3", traversal)
        return traversal

In [48]:
class ParserSettings():
  def __init__(self):
    self.operators = "+-−/÷*×^"
    self.precedence = {'+' : 2, '-' : 2, '−': 2, '*' : 3, '×': 3 , '/' : 3, '÷' : 3, '^': 4}
    self.associativity = {'+' : 'l', '-' : 'l', '−': 'l', '*' : 'l', '×': 'l', '/' : 'l', '÷' : 'l', '^': 'r'}

In [66]:
# Postfix to expression tree.

# Parser
class postfixParser():
  def __init__(self, input_string):

    self.settings = ParserSettings()  
    self.input = input_string
    self.marker = 0
    self.output_stack = Stack()
    self.operator_stack = Stack()

    # This stack will record the marker positions of any whitespace in the input.     
    self.whitespace_position_stack = Stack()

# Since strings are indexed start at 0 the maker only need move to len(input_string)-1. 
    self.input_len = len(self.input)-1 

# Input netural 

  def isWhitespace(self):
    if self.marker <= len(self.input)-1 and self.input[self.marker] in " ":
      self.whitespace_position_stack.push(self.marker)
      self.marker += 1
  
  def isDecimal(self):
    holding_str = ''
    while self.marker <= self.input_len and self.input[self.marker] in "0123456789.":
          holding_str += self.input[self.marker]
          self.marker += 1
    if holding_str != '':
          self.output_stack.push(Node(holding_str))

# Requires postfix input 

  def isBinaryOperation(self):
    if self.input[self.marker] in self.settings.operators:
      op = BinaryTree(self.input[self.marker])
      if type(self.output_stack.peek()) == BinaryTree:
        op.tree_insert_right(self.output_stack.pop(), op.root)
      else:
        op.insert_right(self.output_stack.pop(), op.root)
      
      if type(self.output_stack.peek()) == BinaryTree:
        op.tree_insert_left(self.output_stack.pop(), op.root)
      else:
        op.insert_left(self.output_stack.pop(), op.root)   
      self.output_stack.push(op)

      self.marker += 1
  
  def creatingExp(self):
    while self.marker <= self.input_len:
      self.isWhitespace()
      self.isBinaryOperation()
      self.isDecimal()
    
    return self.output_stack.get_stack()[0]
    

In [67]:
example = postfixParser('34 4.9 + 5 *')

tree = example.creatingExp()

print(tree.print_tree("inorder"))

34+4.9*5


In [ ]:
# Working example of postfix to a binary expression tree.

input_example = 'ab+cde+**' 

# '((ab+)(c(de+)*)*)' becomes '(a+b)*c*(d+e)'

# In infix '((ab+)(c(de+)*)*)' becomes '(a+b)*c*(d+e)'

# Let c be a character in our input expression I.
# Operator Stack
# Holding Stack

# Rule 1 - If c is an operand, check holding stack for pointer T_i, create a one-node tree and push a pointer T_{i+1} to the holding stack.
# Rule 2 - If c is an operator, pop pointers until T_1 is popped. Create a tree with root c and leaves T_1 to T_n. 
# Push a pointer for this tree to the holding stack.   

holding_stack = Stack()
operator_stack = Stack()


input_example = 'ab+cde+**'

# Step 1

marker = 0
c = input_example[0]

# 'a' is an operand. 

T1 = Node('a')
T1.pointer = 1

holding_stack.push(T1)

# Step 2

marker = 1
c = input_example[1]

# 'b' is an operand. 

T2 = Node('b')
T2.pointer = 2

holding_stack.push(T2)

# Step 3 

marker = 2
c = input_example[2]

# '+' is an operator.

O1 = BinaryTree('+')

O1.insert_right(holding_stack.pop())

O1.insert_left(holding_stack.pop())

holding_stack.push(O1)

# Step 3 

marker = 3
c = input_example[3]

# 'c' is a operand.